In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
!pip install catboost
!pip install statsmodels==0.12.1
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import GroupShuffleSplit, train_test_split
from sklearn.metrics import fbeta_score,confusion_matrix,accuracy_score,f1_score,classification_report
pd.set_option("display.max_columns",None)
pd.set_option('display.max_rows', 150)
sns.set_theme(style="darkgrid")

In [ ]:
cd drive/MyDrive/IsBankasi/

#Dataset

*train.csv:* 
  - customer info (id,age,customer seniority, job, etc.) and target 

*monthly_expenditures.csv:*
  - cid, transaction type, transaction amount(aggregated for month), date(month), quantity.

target:
 - whether the customer has married in the next 6 months (last 6 months of year) of their expenditure records.(first 6 months of year.)


In [ ]:
df_train=pd.read_csv("train.csv").drop(columns="tarih").fillna("nan")
df_test=pd.read_csv("test.csv").drop(columns="tarih").fillna("nan")
df_all=pd.concat([df_train,df_test],axis=0)
df_exp=pd.read_csv("monthly_expenditures.csv")
df_exp.tarih=pd.to_datetime(df_exp.tarih.astype(str),format='%Y%m%d').dt.month
#df_exp["ge10K"]=(df_exp.aylik_toplam_tutar > 10000).astype(int)
print("train: ",df_train.shape)
print("test: ",df_test.shape)
print("exp:",df_exp.shape)

train:  (60000, 7)
test:  (40000, 6)
exp: (932144, 5)


# Feature Engineering

1) marking the marginal transactions
- grouping the users with respect to demographics
- marking the marginal transactions (5%) for each demographic group and transaction category

In [ ]:
profile_data=pd.merge(df_exp,df_all,on="musteri").drop(columns="target").groupby(['egitim',"is_durumu",'meslek_grubu','sektor']).quantile(.95).reset_index()[["egitim",	"is_durumu",	"meslek_grubu",	"sektor",	"aylik_toplam_tutar"]]
profile_data.columns=["egitim",	"is_durumu",	"meslek_grubu",	"sektor","group_aylik_toplam_tutar_95_perc"]
df_exp_with_personal_info=pd.merge(df_exp,df_all,on="musteri",how="left").drop(columns=["yas","target","kidem_suresi"])
df_exp_extended=pd.merge(df_exp_with_personal_info,profile_data,on=["egitim","is_durumu","meslek_grubu","sektor"],how="left")
df_exp_extended["is_outlier_transaction"]= (df_exp_extended.group_aylik_toplam_tutar_95_perc < df_exp_extended.aylik_toplam_tutar).astype(int)
df_exp_extended= df_exp_extended.drop(columns=["egitim"	,"is_durumu",	"meslek_grubu"	,"group_aylik_toplam_tutar_95_perc"])
df_exp=df_exp_extended

In [ ]:

def lin_regression(df):
  try:
    y=df['aylik_toplam_tutar'].values
    x=df['tarih'].values
    if (len(y)==1):
      slope= np.nan
    else:  
      m_x=np.mean(x)
      m_y=np.mean(y)
      slope=np.sum((x-m_x)*(y-m_y)) / np.sum((x-m_x)**2)
  except:
        slope = np.nan

  return slope

2) Aggregate Variables for each customer (all sectors)


In [ ]:
agg_vars_musteri_level= df_exp.groupby(['musteri']).agg({'islem_adedi':['mean','std','sum','max'],
                                                           'aylik_toplam_tutar':['mean','std','sum','max','min'],
                                                         }).reset_index()
agg_vars_musteri_level.columns = ["_".join(x) if len(x[1]) > 1 else x[0] for x in agg_vars_musteri_level.columns.ravel()]
agg_vars_musteri_level_cols=[x  for x in agg_vars_musteri_level.columns if "_" in x]

#calculating the trend slope of total amount of transactions
harcama_trend_tekil=df_exp.groupby(by=["musteri","tarih"]).sum().reset_index().groupby(by=['musteri']).apply(lin_regression).reset_index().rename(columns={0:"aylik_toplam_tutar_SUM_slope"})


#amount of each month aggreated then transposed 
musteri_tarih_level= df_exp.groupby(["musteri","tarih"]).agg({'aylik_toplam_tutar': 'sum'})
musteri_level= df_exp.groupby(['musteri']).agg({'aylik_toplam_tutar': 'sum'})
musteri_tarih_level=musteri_tarih_level.div(musteri_level, level='musteri').multiply(100).reset_index().rename(columns={"aylik_toplam_tutar":"att_perc"})
musteri_tarih_level_tekil=musteri_tarih_level.pivot(index='musteri', columns='tarih', values="att_perc").reset_index()
musteri_tarih_level_tekil.columns=[x+"_att_ratio" if str(x)[0].isupper() else x for x in musteri_tarih_level_tekil.columns ]

#transaction sector counts
sector_counts=df_exp.groupby(["musteri","tarih"])["sektor"].count().groupby("musteri").max()


agg_vars_musteri_level= pd.merge(agg_vars_musteri_level,harcama_trend_tekil,how="outer",on="musteri")
agg_vars_musteri_level= pd.merge(agg_vars_musteri_level,musteri_tarih_level_tekil,how="outer",on="musteri")
agg_vars_musteri_level= pd.merge(agg_vars_musteri_level,sector_counts,how="outer",on="musteri")

3) Aggregate Variables for each customer (sector breakdown)


In [ ]:
agg_vars_sektor_level=df_exp.groupby(['musteri','sektor']).agg({'islem_adedi':['mean','std','sum','max'],
                                                           'aylik_toplam_tutar':['mean','std','sum','max','count'],
                                                           'is_outlier_transaction':['mean','sum'],
                                                   'tarih':['count']}).reset_index()
agg_vars_sektor_level.columns = ["_".join(x) if len(x[1]) > 1 else x[0] for x in agg_vars_sektor_level.columns.ravel()]
agg_vars_sektor_level_cols=[x  for x in agg_vars_sektor_level.columns if "_" in x]
agg_vars_sektor_level_tekil=agg_vars_sektor_level.pivot(index='musteri', columns='sektor', values=agg_vars_sektor_level_cols).reset_index()
agg_vars_sektor_level_tekil.columns = ["_".join(x) if len(x[1]) > 1 else x[0] for x in agg_vars_sektor_level_tekil.columns.ravel()]


sektor_trends=df_exp.groupby(by=['musteri','sektor']).apply(lin_regression).reset_index().rename(columns={0:"slope"})
sektor_trends_tekil=sektor_trends.pivot(index='musteri', columns='sektor', values='slope').reset_index()
sektor_trends_tekil.columns = [col + "_slope" if col.isupper() else col for col in sektor_trends_tekil.columns] 
#adding sector-level trend analysis to other sector-level features
agg_vars_sektor_level_tekil=pd.merge(sektor_trends_tekil,agg_vars_sektor_level_tekil,how="outer",on="musteri")


#ratio of sector / total expenditures
sector_ratios=agg_vars_sektor_level_tekil.filter(like="aylik_toplam_tutar_sum").div(agg_vars_musteri_level.aylik_toplam_tutar_sum, axis=0)
sector_ratios.columns = [col + "_ratio" for col in sector_ratios.columns] 
agg_vars_sektor_level_tekil=pd.concat([agg_vars_sektor_level_tekil,sector_ratios],axis=1)


4) Finding mostly spend sector for eaqch customer

In [ ]:

max_spent_transaction=df_exp.sort_values(by=["musteri","aylik_toplam_tutar"],ascending=False)\
        .drop_duplicates(subset="musteri",keep="first").drop(columns=["islem_adedi"])\
        .rename(columns={"sektor":"MST_sektor_name","aylik_toplam_tutar":"MST_sektor_att","tarih":"MST_sektor_month"})


max_spent_ever=df_exp.groupby(["musteri","sektor"]).sum().reset_index().sort_values(by=["musteri","aylik_toplam_tutar"]).drop_duplicates(subset="musteri",keep="last")\
                .drop(columns=["islem_adedi"])\
                .rename(columns={"sektor":"MSE_sektor_name","aylik_toplam_tutar":"MSE_sektor_att","tarih":"MSE_sektor_month"})

5) Getting the largest difference in consecutive months (assuming spikes may indicate wedding etc.)

In [ ]:
df_musteri_tarih_total=df_exp.groupby(by=['musteri','tarih']).sum().reset_index()
df_musteri_tarih_total["difference"]= df_musteri_tarih_total.sort_values(by=['musteri', 'tarih']).groupby(by=['musteri'])['aylik_toplam_tutar'].diff(periods=1)
df_delta= df_musteri_tarih_total.groupby("musteri").agg({'difference':['max','min']}).reset_index()
df_delta.columns = ["_".join(x) if len(x[1]) > 1 else x[0] for x in df_delta.columns.ravel()]

In [ ]:
expenditure_features=pd.merge(agg_vars_sektor_level_tekil,agg_vars_musteri_level,how="outer",on="musteri")
expenditure_features=pd.merge(expenditure_features,max_spent_transaction,how="outer",on="musteri")
expenditure_features=pd.merge(expenditure_features,max_spent_ever,how="outer",on="musteri")
expenditure_features=pd.merge(expenditure_features,df_delta,how="outer",on="musteri")
df_merged=pd.merge(df_train,expenditure_features,how="inner",on="musteri")

#Feature Elimination

-removing highly correlated features

In [ ]:
# Create correlation matrix
corr_matrix = df_merged.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df_merged.drop(to_drop, axis=1, inplace=True)

In [ ]:

X=df_merged.drop(columns="target").drop(columns="musteri").fillna(0)#[important_features]

categorical_features_indices = np.where(X.dtypes== np.object )[0]
categorical_features= X.columns[categorical_features_indices]
numerical_features= list(set(X.columns) - set(categorical_features))

X_with_dummies= pd.get_dummies(X,columns=categorical_features)

y=df_merged.target


X[categorical_features]=X[categorical_features].fillna("nan") #catboost'a girmeden önce categoricallardaki tüm NaN değerler string'e çevrilmelidir.
for f in categorical_features:
  X[f]=X[f].astype(str)

for feature in categorical_features:
  X[feature] = pd.Series(X[feature], dtype="category")

categorical_features_indices = np.where(X.dtypes== "category" )[0]
categorical_features= X.columns[categorical_features_indices]
numerical_features= list(set(X.columns) - set(categorical_features))

 


# LightGBM model

In [ ]:

import lightgbm as lgb
Xd= pd.get_dummies(X,columns=categorical_features)
Xd_train, Xd_test, yd_train, yd_test = train_test_split(Xd, y, test_size=0.15, random_state=4,stratify=y)
Xd_train, Xd_val, yd_train, yd_val = train_test_split(Xd_train, yd_train, test_size=0.2, random_state=4,stratify=yd_train)
lightmodel = lgb.LGBMClassifier(objective = "binary",class_weight="balanced")
lightmodel.fit(  X = Xd_train,  y = yd_train,    eval_set=(Xd_val, yd_val),categorical_feature = 'auto',verbose=False)




light_predictions_test=lightmodel.predict(Xd_test)

light_acc_test = accuracy_score(yd_test, light_predictions_test)

print("Lightgbm Accuracy:"+str(light_acc_test))

print("Confusion Matrix : \n", confusion_matrix(yd_test, light_predictions_test))
print(classification_report(yd_test, light_predictions_test, digits=4))

In [ ]:
lgb.plot_importance(lightmodel,max_num_features=15)

In [ ]:
 #Since catboost can work with categorical values without OHE , we proceed so
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=4,stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=4,stratify=y_train)



print(f"x_train y_train shape{X_train.shape}{y_train.shape}")
print(f"x_val y_val shape{X_val.shape}{y_val.shape}")
print(f"x_test y_test shape{X_test.shape}{y_test.shape}")

x_train y_train shape(40800, 214)(40800,)
x_val y_val shape(10200, 214)(10200,)
x_test y_test shape(9000, 214)(9000,)


#Oversampling with SMOTE
-Original Ratio: 1:39
-Oversampled: 1:1

In [ ]:
from collections import Counter
from numpy.random import RandomState
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTENC
print('Original dataset shape (%s, %s)' % X_train.shape)

print('Original dataset samples per class {}'.format(Counter(y_train)))


sm = SMOTENC(random_state=42, categorical_features=categorical_features_indices,k_neighbors=9)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset samples per class {}'.format(Counter(y_res)))


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Original dataset shape (40800, 147)
Original dataset samples per class Counter({0: 39141, 1: 1659})


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Resampled dataset samples per class Counter({0: 39141, 1: 39141})


# CatBoost model with validation

In [ ]:
catmodel = CatBoostClassifier(
    random_seed=42,
    logging_level='Verbose',
    iterations=200,
    use_best_model=True,
    class_weights=[1,10],
    l2_leaf_reg=5,
    
    )


catmodel.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_val, y_val),
)

In [ ]:
from sklearn.metrics import fbeta_score,confusion_matrix,accuracy_score,f1_score,classification_report

cat_predictions_test=catmodel.predict(X_test)

cat_acc_test = accuracy_score(y_test, cat_predictions_test)

print("Catboost Accuracy:"+str(cat_acc_test))

print("Confusion Matrix : \n", confusion_matrix(y_test, cat_predictions_test))
print(classification_report(y_test, cat_predictions_test, digits=4))

In [ ]:
from catboost.utils import get_roc_curve
import sklearn
from sklearn import metrics

eval_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)
curve = get_roc_curve(catmodel, eval_pool)
(fpr, tpr, thresholds) = curve
roc_auc = sklearn.metrics.auc(fpr, tpr)

metrics = catmodel.eval_metrics(
    data=eval_pool,
    metrics=['Accuracy',"Precision","Recall"],
)


In [ ]:

from sklearn import metrics

metrics.roc_auc_score(y_test, cat_predictions_test)

In [ ]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, cat_predictions_test)
metrics.auc(fpr, tpr)

In [ ]:
plt.figure(figsize=(8, 4))
lw = 2

plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc, alpha=0.5)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', alpha=0.5)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(True)
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('Receiver operating characteristic', fontsize=20)
plt.legend(loc="lower right", fontsize=16)


from catboost.utils import get_fpr_curve
from catboost.utils import get_fnr_curve

(thresholds, fpr) = get_fpr_curve(curve=curve)
(thresholds, fnr) = get_fnr_curve(curve=curve)

plt.figure(figsize=(8, 4))

lw = 2

plt.plot(thresholds, fpr, color='blue', lw=lw, label='FPR', alpha=0.5)
plt.plot(thresholds, fnr, color='green', lw=lw, label='FNR', alpha=0.5)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(True)
plt.xlabel('Threshold', fontsize=16)
plt.ylabel('Error Rate', fontsize=16)
plt.title('FPR-FNR curves', fontsize=20)
plt.legend(loc="lower left", fontsize=16)

plt.show()

In [ ]:
test_results=pd.concat([pd.concat([df_merged.iloc[X_test.index].musteri,X_test,y_test,],axis=1).reset_index(drop=True),pd.Series(cat_predictions_test,name="prediction")],axis=1)

In [ ]:
missed= test_results[["musteri","yas","kidem_suresi","egitim","is_durumu","meslek_grubu","target","prediction"]].query("target==1 and prediction==0")

In [ ]:
df_submission=pd.DataFrame(data={"musteri":df_test.musteri,"target":catmodel.predict(df_test_merged)})

In [ ]:
df_submission.to_csv("13_02_2021_catmodel_v2.csv",index=False)

In [ ]:
catmodel.predict(df_test_merged)